In [ ]:
# Yuliya Akchurina
# CISC6210 - Natural Language Processing 
# Homework2 - Ngrams

In [1]:
import urllib
from urllib import request
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from nltk import FreqDist
import copy 

url = "https://storm.cis.fordham.edu/~yli/data/MyShakespeare.txt"

In [2]:
def upload_preprocess(url):
    # Uploading and preprocessing text. Tokenise words. Keep the punctuation and special characters.
    #import text from a url. Returns string variable with all text in it. String lenght 51896
    raw = request.urlopen(url).read().decode('utf8')

    #convert to lower case. Returns string variable 
    raw_lower = raw.lower()

    # split text into separte words. Returns list of lowercase word tokens, all punctuation included. 11320 tokens with punctuation.  
    word_tokens = nltk.word_tokenize(raw_lower)
    return word_tokens

def get_vocab(word_tokens):
    vocab_list = list(set(word_tokens))
    return vocab_list

def size_vocab(word_tokens):
    vocab_size = len(list(set(word_tokens)))
    return vocab_size 

def len_text(word_tokens):
    length_text = len(word_tokens)
    return length_text

# Obtain list of Ngrams. Takes in n - number of n in ngrams, and text - list of toknized words
def get_ngram(text, n):
    ngram_list = list(ngrams(text, n))
    return ngram_list

def len_ngram(ngram_list):
    return len(ngram_list)

def ngram_freq(gram):
    #returns frequency distribution of ngrams and a tuple for the most frequent ngram 
    fdist = FreqDist(gram) 
    return fdist

def input_ngram(context, word):
    
    context = input(f"Please enter a beginning of Ngram string length {n-1} :   ")
    word = input("Please enter the last word of the NGram:   ")
    
    context = context.lower()
    word = word.lower()
    
    full_ngram = context + word
    
    #context_token = nltk.word_tokenize(context)
    #last_word_token = nltk.word_tokenize(word)
    
    #full_ngram = copy.deepcopy(context_token)
    #full_ngram.extend(last_word_token)
    return context, word, full_ngram


def process_text(context, word):
    #takes in strings, returns tokenized lists 
    context = context.lower()
    word = word.lower()

    context_token = nltk.word_tokenize(context)
    last_word_token = nltk.word_tokenize(word)
    return context_token, last_word_token


def get_prob(context, word, n):
    # takes in strings, returns probabiity
    """ Prob(word | context) = (count(context+word)) / count(context)) """
    
    if n == 1:
        probability = 1/vocab_size
    
    else:
    
        # create context ngrams, n-1 ngrams and word ngrams from given text url
        context_gram = get_ngram(upload_preprocess(url), n-1)
        freqdist_context_gram = ngram_freq(context_gram)
        
        word_gram = get_ngram(upload_preprocess(url), 1)
        freqdist_word_gram = ngram_freq(word_gram)     

        n_gram = get_ngram(upload_preprocess(url), n)
        freqdist_n_gram = ngram_freq(n_gram)
        
        context_token, last_word_token = process_text(context, word)
        
        # check if context and word occur in text 
        context_tuple = tuple(context_token)
        word_tuple = tuple(last_word_token)
        full_ngram = []
        full_ngram = context_token + last_word_token
        full_ngram_tuple = tuple(full_ngram)

        # Calculate counts and probabilities 
        count_ngram = freqdist_n_gram.get(full_ngram_tuple)
        count_context = freqdist_context_gram.get(context_tuple)

        # check context  
        if context_tuple in freqdist_context_gram.keys():
            print("Context exists")
            #print(freqdist_context_gram.get(context_tuple))
            context_seen = True
        else:
            print("Context not found")
            context_seen = False

        # check word
        if word_tuple in freqdist_word_gram.keys():
            print("Word exists")
            #print(freqdist_word_gram.get(word_tuple))
            word_seen = True
        else:
            print("Word not found")
            probability_word = 1/(size_vocab(upload_preprocess(url)) + 1)
            word_seen = False
        if full_ngram_tuple in freqdist_n_gram.keys():
            print("Ngram exists")
            #print(freqdist_word_gram.get(word_tuple))
            ngram_seen = True
        else:
            print("Ngram not found")
            ngram_seen = False

        #if context_seen and word_seen:
            #probability = count_ngram/count_context
            
        if context_seen and word_seen and ngram_seen:
            probability = count_ngram/count_context

        elif context_seen and word_seen and not ngram_seen:
            # if the context and word exist in the text but the ngram of context+word does not exist in the text. 
            count_ngram = 0 + 1
            probability = count_ngram/count_context

        elif context_seen and not word_seen:
            print("The context is the text. The word is not in the text.")
            count_ngram = 0
            probability = count_ngram/count_context

        elif not context_seen and word_seen:
            #If you encounter a novel context, and the word exists in the vocabulary, 
            #the probability of any given word should be 1/the size of the vocab.
            print("The context is not in the text. The word is in the text.")
            probability = 1/size_vocab(upload_preprocess(url))

        elif not context_seen and not word_seen:
            #If the word is unknown, the probability of the unknown word should be 1/(the size of vocab+1)
            print("The context is not in the text. The word is not in the text.")
            probability = 1/(size_vocab(upload_preprocess(url)) + 1)

        else: 
            print(" Oops! Something went wrong when calculating probabilities. ")

    print(f"Probability of the word: '{word}' given context: '{context}' is = {probability}")

    return probability


def generate_text(context, max_length):
    context_lower = context.lower()      #convert to lowercase
    context_token = nltk.word_tokenize(context_lower)   #tokenize 
    k = len(context_token)   #context token count

    # get all k+1 size ngrams
    plusOne_ngram = get_ngram(upload_preprocess(url), k+1)
    freqdist_plusOne_ngram = ngram_freq(plusOne_ngram) 
    
    # nested list of length 6320
    dictlist = list(freqdist_plusOne_ngram.items())
    
    ngram_start_list = []
    frequency_sent_list = []
    
    search_token = copy.deepcopy(context_token)
    result_sentence = copy.deepcopy(context_token)
    
    while len(result_sentence) < max_length:  
        max_match_ngram = []
        ngram_start_list =[]
        
        for i in range(len(dictlist)):
            if (' '.join(dictlist[i][0])).startswith(' '.join(search_token)):
                #print(dictlist[i])
                ngram_start_list.append(dictlist[i])

        # update the context sentence with new word until the length of the sentense reached max_length
        max_match_ngram = list(max(ngram_start_list, key=lambda x: x[1])[0])
        #print(f"max_match_ngram {max_match_ngram}")
        
        result_sentence.append(max_match_ngram[len(max_match_ngram)-1])
        frequency_sent_list.append(max(ngram_start_list, key=lambda x: x[1])[1])   #frequency
        
        #update search token
        l = len(result_sentence)
        search_token = []
        search_token.extend([result_sentence[l-2],result_sentence[l-1]])
    
    return(result_sentence)
    #return(result_sentence, frequency_sent_list)

def perplexity(text, n):
    text_token = nltk.word_tokenize(text)
    text_len = (len(text_token))
    prob_list = []

    for i in range(0, text_len - 2):
        calculated_prob = get_prob([text_token[i], text_token[i+1]], [text_token[i+2]], n)
        prob_list.append(calculated_prob)

    product = 1
    for item in prob_list:
        product = product * item
        get_perplexity = (product)**(-1/text_len)
    return get_perplexity
    

In [4]:
# Run all functions 
n = 3

word_tokenize = upload_preprocess(url)
vocab_list = get_vocab(word_tokenize)
vocab_size = size_vocab(word_tokenize)
text_length = len_text(word_tokenize)
ngram_list = get_ngram(word_tokenize, n)
ngram_list_len = len_ngram(ngram_list)

In [5]:
# testing Pobability function

wordone = "our business"
wordtwo = "is"
prob_test = get_prob(wordone, wordtwo, n=3)

print(prob_test)

wordone = "our"
wordtwo = "business"
prob_test2 = get_prob(wordone, wordtwo, n=2)

print(prob_test2)

wordone = " "
wordtwo = "business"
prob_test3 = get_prob(wordone, wordtwo, n=1)

print(prob_test3)

Context exists
Word exists
Ngram exists
Probability of the word: 'is' given context: 'our business' is = 1.0
1.0
Context exists
Word exists
Ngram exists
Probability of the word: 'business' given context: 'our' is = 0.03225806451612903
0.03225806451612903
Probability of the word: 'business' given context: ' ' is = 0.0006978367062107466
0.0006978367062107466


In [6]:
### Generate a sentence length 30 with seed words "our business" based on the 3gram modeL ###
context_seed = "our business"
generated_sentence = generate_text(context_seed, 30)
print(f"The 30 word generated sentence from three-gram model is: \n\n{generated_sentence}")
print(f"\nLength of the sentence {len(generated_sentence)}")

"""['our', 'business', 'is', 'not', 'unknown', 'to', 'the', 'capitol', ';', 'who', 
"'s", 'like','to', 'rise', ',', 'who', 'is', 'the', 'sink', 'o', 
"'", 'the', 'body', ',', 'idle', 'and', 'unactive',',', 'still', 'cupboarding']"""

The 30 word generated sentence from three-gram model is: 

['our', 'business', 'is', 'not', 'unknown', 'to', 'the', 'capitol', ';', 'who', "'s", 'like', 'to', 'rise', ',', 'who', 'is', 'the', 'sink', 'o', "'", 'the', 'body', ',', 'idle', 'and', 'unactive', ',', 'still', 'cupboarding']

Length of the sentence 30


In [58]:
unigrams = get_ngram(word_tokenize, n=1)     #11320
bigrams = get_ngram(word_tokenize, n=2)    #11319
trigrams = get_ngram(word_tokenize, n=3)    #11318

In [20]:
freqdist_unigram = ngram_freq(unigrams)
freqdist_bigram = ngram_freq(bigrams)
freqdist_trigram = ngram_freq(trigrams)
print(freqdist_unigram.most_common(10))
print(freqdist_bigram.most_common(10))
print(freqdist_trigram.most_common(10))

[((',',), 842), ((':',), 506), (('.',), 397), (('the',), 369), (('you',), 219), (('and',), 192), (('i',), 187), (('to',), 178), ((';',), 160), (('a',), 117)]
[((',', 'and'), 71), (('citizen', ':'), 58), ((',', 'i'), 53), (('marcius', ':'), 53), (('first', 'citizen'), 46), ((',', 'you'), 39), (('.', 'marcius'), 37), (('.', 'first'), 36), ((',', 'the'), 36), (('menenius', ':'), 36)]
[(('first', 'citizen', ':'), 46), (('.', 'marcius', ':'), 37), (('.', 'virgilia', ':'), 28), (('.', 'first', 'citizen'), 26), (('.', 'valeria', ':'), 24), (('.', 'menenius', ':'), 20), (('o', "'", 'the'), 18), (('.', 'lartius', ':'), 16), (('first', 'senator', ':'), 15), (('?', 'first', 'citizen'), 14)]


In [60]:
# Perplexity for trigrams 
# n=3 
text = "make you a sword for me" 
text_token = nltk.word_tokenize(text.lower())

probability_list = []

for i in range(0, int(len(text_token)-2)):
    context_tuple = tuple([text_token[i], text_token[i+1]])
    #word_tuple = tuple(text_token[i+2])
 
    trigram_tuple = tuple([text_token[i], text_token[i+1], text_token[i+2]])

    # Calculate counts and probabilities 
    count_ngram = freqdist_trigram.get(trigram_tuple)
    count_context = freqdist_bigram.get(context_tuple)
    
    if count_context is not None and count_ngram is not None:
        probability = count_ngram/count_context      
    
    elif count_context is None:
        probability = 1/vocab_size
           
    elif count_ngram is None:
        probability = 1/(vocab_size+1)
      
    probability_list.append(probability)

product = 1
for item in probability_list:
    product = product*item

    
trigram_perplexity = product**(-1/len(text_token))
print(f"Trigram Perplexity = {trigram_perplexity}")  # Trigram Perplexity 16.261963246864724

Trigram Perplexity = 16.261963246864724


In [62]:
# Perplexity for bigrams 
# n=2 
text = "make you a sword for me" 
text_token = nltk.word_tokenize(text.lower())

probability_list = []

for i in range(0, int(len(text_token)-1)):
    #context_tuple = tuple(text_token[i])
    #word_tuple = tuple(text_token[i+1])
    context = text_token[i]
    word = text_token[i+1]

    bigram_tuple = tuple([text_token[i], text_token[i+1]])

    # Calculate counts and probabilities 
    count_ngram = freqdist_bigram.get(bigram_tuple)
    count_context = freqdist_unigram.get(context)
    
    if count_context is not None and count_ngram is not None:
        probability = count_ngram/count_context
        
    
    elif count_context is None:
        probability = 1/vocab_size
        
    
    elif count_ngram is None:
        probability = 1/(vocab_size+1)
      
    probability_list.append(probability)
    
#print(probability_list)

product = 1
for item in probability_list:
    product = product*item

bigram_perplexity = product**(-1/len(text_token))
print(f"Bigram Perplexity = {bigram_perplexity}")   # Bigram Perplexity 426.78108055853585

Bigram Perplexity = 426.78108055853585


In [63]:
# Perplexity for unigrams 
# n=1
text = "make you a sword for me"    
text = nltk.word_tokenize(context.lower())

text_len = (len(text))
prob_list = []
n=1

for word in text:
    calculated_prob = 1/vocab_size
    prob_list.append(calculated_prob)

product = 1
for item in prob_list:
    product = product * item
unigram_perplexity = (product)**(-1/text_len)

print(f"Unigram perplexity = {unigram_perplexity} ")  # Unigram perplexity 1432.9999999999995 

Unigram perplexity = 1433.0 
